Check if Data is ready for Training

### Classes in MVTEC_AD dataset

In [9]:
from pathlib import Path

root = Path("/Users/ajayyy/Desktop/Deep_Learning/Smart-Quality-Inspection-System/data/raw/MVTEC_AD/mvtec_anomaly_detection")
classes = set()
for category in root.iterdir():
    if not category.is_dir():
        continue
    test_dir = category / "test"
    if test_dir.exists():
        for defect_type in test_dir.iterdir():
            if defect_type.is_dir() and defect_type.name != "good":
                classes.add(defect_type.name)

print(f"Total defect classes: {len(classes)}")
print(sorted(classes))


FileNotFoundError: [Errno 2] No such file or directory: '/Users/ajayyy/Desktop/Deep_Learning/Smart-Quality-Inspection-System/data/raw/MVTEC_AD/mvtec_anomaly_detection'

# Koklektor Dataset

In [3]:
from pathlib import Path

root = Path("/Users/ajayyy/Desktop/Deep_Learning/Smart-Quality-Inspection-System/data/raw/KolektorSDD/KolektorSDD2")
train_dir = root / "train"
test_dir = root / "test"

def count_defects(folder):
    defects = 0
    total = 0
    for img in folder.glob("*.png"):
        if img.name.endswith("_GT.png"):
            continue
        total += 1
        mask = folder / f"{img.stem}_GT.png"
        if mask.exists():
            defects += 1
    return total, defects

train_total, train_defects = count_defects(train_dir)
test_total, test_defects = count_defects(test_dir)

print(f"TRAIN: {train_defects}/{train_total} defective")
print(f"TEST: {test_defects}/{test_total} defective")


TRAIN: 2331/2333 defective
TEST: 1004/1004 defective


In [1]:
import os
import shutil
from pathlib import Path
import pandas as pd

# ==============================
# CONFIGURATION
# ==============================
RAW_DATASET_DIR = Path("/Users/ajayyy/Desktop/Deep_Learning/Smart-Quality-Inspection-System/data/raw/MVTEC_AD/mvtec_anomaly_detection")  # <-- change this
OUTPUT_DIR = Path("/Users/ajayyy/Desktop/Deep_Learning/Smart-Quality-Inspection-System/data/processed")

# Create output structure
(OUTPUT_DIR / "images/train").mkdir(parents=True, exist_ok=True)
(OUTPUT_DIR / "images/val").mkdir(parents=True, exist_ok=True)
(OUTPUT_DIR / "masks/val").mkdir(parents=True, exist_ok=True)
(OUTPUT_DIR / "metadata").mkdir(parents=True, exist_ok=True)

metadata = []

# ==============================
# ITERATE OVER ALL CATEGORIES
# ==============================
for category in sorted(os.listdir(RAW_DATASET_DIR)):
    category_path = RAW_DATASET_DIR / category
    if not category_path.is_dir():
        continue

    print(f"Processing category: {category}")

    # --- TRAIN (GOOD IMAGES) ---
    train_good_dir = category_path / "train" / "good"
    if train_good_dir.exists():
        for img_file in sorted(train_good_dir.glob("*")):
            new_name = f"{category}_good_train_{img_file.name}"
            dest = OUTPUT_DIR / "images/train" / new_name
            shutil.copy(img_file, dest)
            metadata.append({
                "category": category,
                "subset": "train",
                "type": "good",
                "image": new_name,
                "mask": None
            })

    # --- TEST (GOOD + DEFECTIVE) ---
    test_dir = category_path / "test"
    gt_dir = category_path / "ground_truth"

    if test_dir.exists():
        for defect_type in sorted(os.listdir(test_dir)):
            defect_dir = test_dir / defect_type
            if not defect_dir.is_dir():
                continue

            for img_file in sorted(defect_dir.glob("*")):
                new_name = f"{category}_{defect_type}_{img_file.name}"
                dest = OUTPUT_DIR / "images/val" / new_name
                shutil.copy(img_file, dest)

                mask_file = None
                if defect_type != "good":  # Only defective images have masks
                    mask_dir = gt_dir / defect_type
                    if mask_dir.exists():
                        base_name = img_file.stem
                        possible_masks = list(mask_dir.glob(f"{base_name}*"))
                        if possible_masks:
                            mask_file = possible_masks[0]
                            new_mask_name = f"{category}_{defect_type}_{mask_file.name}"
                            mask_dest = OUTPUT_DIR / "masks/val" / new_mask_name
                            shutil.copy(mask_file, mask_dest)
                            mask_file = new_mask_name

                metadata.append({
                    "category": category,
                    "subset": "val",
                    "type": defect_type,
                    "image": new_name,
                    "mask": mask_file
                })

# ==============================
# SAVE METADATA
# ==============================
df = pd.DataFrame(metadata)
df.to_csv(OUTPUT_DIR / "metadata" / "mapping.csv", index=False)
print("\nOrganization complete!")
print(f"Images and masks saved under: {OUTPUT_DIR}")
print(f"Metadata CSV: {OUTPUT_DIR / 'metadata/mapping.csv'}")


Processing category: bottle
Processing category: cable
Processing category: capsule
Processing category: carpet
Processing category: grid
Processing category: hazelnut
Processing category: leather
Processing category: metal_nut
Processing category: pill
Processing category: screw
Processing category: tile
Processing category: toothbrush
Processing category: transistor
Processing category: wood
Processing category: zipper

Organization complete!
Images and masks saved under: /Users/ajayyy/Desktop/Deep_Learning/Smart-Quality-Inspection-System/data/processed
Metadata CSV: /Users/ajayyy/Desktop/Deep_Learning/Smart-Quality-Inspection-System/data/processed/metadata/mapping.csv


## Sanity Checks

**imports**

In [2]:
import os
import cv2
import random
import matplotlib.pyplot as plt

In [3]:
DATASET_PATH = "/Users/ajayyy/Desktop/Deep_Learning/Smart-Quality-Inspection-System/data/processed"

**Verify every defective image has a corresponding mask**

In [6]:
defective_missing_masks = []

for category in categories:
    gt_path = os.path.join(DATASET_PATH, category, "ground_truth")
    test_path = os.path.join(DATASET_PATH, category, "test")
    if not os.path.exists(gt_path):
        continue

    for defect_type in os.listdir(gt_path):
        gt_files = sorted(os.listdir(os.path.join(gt_path, defect_type)))
        test_files = sorted(os.listdir(os.path.join(test_path, defect_type)))

        gt_basenames = [os.path.splitext(f)[0] for f in gt_files]
        test_basenames = [os.path.splitext(f)[0] for f in test_files]

        missing = [f for f in test_basenames if f not in gt_basenames]
        if missing:
            defective_missing_masks.extend([(category, defect_type, m) for m in missing])

if defective_missing_masks:
    print("\n⚠️ Missing masks for these images:")
    for item in defective_missing_masks[:10]:
        print(item)
else:
    print("\n All defective images have corresponding masks!")


 All defective images have corresponding masks!


## checking labels 

In [12]:
import os
import cv2
import random

# === PATHS ===
base_dir = "/Users/ajayyy/Desktop/Deep_Learning/Smart-Quality-Inspection-System/data/processed"
images_dir = os.path.join(base_dir, "images/val")
labels_dir = os.path.join(base_dir, "labels/val")
output_dir = os.path.join(base_dir, "sanity_check_outputs")

os.makedirs(output_dir, exist_ok=True)

num_samples = 10 

# === FUNCTION TO DRAW BOXES ===
def draw_yolo_boxes(img_path, label_path, save_path):
    img = cv2.imread(img_path)
    if img is None:
        return False

    h, w = img.shape[:2]
    if not os.path.exists(label_path):
        return False

    with open(label_path, "r") as f:
        lines = f.readlines()

    for line in lines:
        parts = line.strip().split()
        if len(parts) != 5:
            continue
        cls, x, y, bw, bh = map(float, parts)
        x1 = int((x - bw / 2) * w)
        y1 = int((y - bh / 2) * h)
        x2 = int((x + bw / 2) * w)
        y2 = int((y + bh / 2) * h)
        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 0, 255), 2)
        cv2.putText(img, f"defect", (x1, y1 - 5),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)

    cv2.imwrite(save_path, img)
    return True

# === RANDOMLY CHECK SOME LABELS ===
label_files = [f for f in os.listdir(labels_dir) if f.endswith(".txt")]
print(f"Found {len(label_files)} label files.")

sample_files = random.sample(label_files, min(num_samples, len(label_files)))

for lf in sample_files:
    label_path = os.path.join(labels_dir, lf)
    img_name = lf.replace(".txt", ".png")
    img_path = os.path.join(images_dir, img_name)
    if not os.path.exists(img_path):
        img_name = lf.replace(".txt", ".jpg")
        img_path = os.path.join(images_dir, img_name)
        if not os.path.exists(img_path):
            continue
    save_path = os.path.join(output_dir, img_name)
    success = draw_yolo_boxes(img_path, label_path, save_path)
    if success:
        print(f"✓ Visualized: {img_name}")
    else:
        print(f"⚠️ Skipped: {img_name}")

print(f"\nSanity check complete! Labeled samples saved to:\n{output_dir}")


Found 1258 label files.
✓ Visualized: carpet_color_003.png
✓ Visualized: tile_rough_006.png
✓ Visualized: wood_combined_005.png
✓ Visualized: zipper_fabric_border_014.png
✓ Visualized: bottle_broken_large_001.png
✓ Visualized: hazelnut_hole_008.png
✓ Visualized: wood_color_002.png
✓ Visualized: capsule_squeeze_008.png
✓ Visualized: zipper_fabric_interior_011.png
✓ Visualized: zipper_rough_016.png

Sanity check complete! Labeled samples saved to:
/Users/ajayyy/Desktop/Deep_Learning/Smart-Quality-Inspection-System/data/processed/sanity_check_outputs


## Organising Kolektor Dataset

In [14]:
import os
import shutil
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

base_dir = "/Users/ajayyy/Desktop/Deep_Learning/Smart-Quality-Inspection-System/data"
raw_dir = os.path.join(base_dir, "raw/KolektorSDD/KolektorSDD2")
processed_dir = os.path.join(base_dir, "processed")

images_dir = os.path.join(processed_dir, "images2")
masks_dir = os.path.join(processed_dir, "masks2")
os.makedirs(images_dir, exist_ok=True)
os.makedirs(masks_dir, exist_ok=True)

def copy_file(src, dst):
    try:
        shutil.copy2(src, dst)
    except Exception as e:
        print(f"Error copying {src}: {e}")

def organize_kolektor(raw_split, split_name):
    split_path = os.path.join(raw_dir, raw_split)
    img_out = os.path.join(images_dir, split_name)
    mask_out = os.path.join(masks_dir, split_name)
    os.makedirs(img_out, exist_ok=True)
    os.makedirs(mask_out, exist_ok=True)

    files = [f for f in os.listdir(split_path) if f.lower().endswith((".png", ".jpg"))]

    tasks = []
    with ThreadPoolExecutor(max_workers=8) as executor:
        for f in tqdm(files, desc=f"Organizing {split_name}"):
            src = os.path.join(split_path, f)
            if "_gt" in f.lower():
                dst = os.path.join(mask_out, f.replace("_GT", "").replace("_gt", ""))
            else:
                dst = os.path.join(img_out, f)
            tasks.append(executor.submit(copy_file, src, dst))

        for task in as_completed(tasks):
            _ = task.result()

    print(f"{split_name} done — {len(files)} files processed.")

organize_kolektor("train", "train")
organize_kolektor("test", "val")


Organizing train: 100%|██████████| 4878/4878 [00:00<00:00, 93541.70it/s]


train done — 4878 files processed.


Organizing val: 100%|██████████| 2082/2082 [00:00<00:00, 161456.59it/s]


val done — 2082 files processed.


In [15]:
import os

print("train images:", len(os.listdir("/Users/ajayyy/Desktop/Deep_Learning/Smart-Quality-Inspection-System/data/processed/images2/train")))
print("train masks:", len(os.listdir("/Users/ajayyy/Desktop/Deep_Learning/Smart-Quality-Inspection-System/data/processed/masks2/train")))
print("val images:", len(os.listdir("/Users/ajayyy/Desktop/Deep_Learning/Smart-Quality-Inspection-System/data/processed/images2/val")))
print("val masks:", len(os.listdir("/Users/ajayyy/Desktop/Deep_Learning/Smart-Quality-Inspection-System/data/processed/masks2/val")))


train images: 2439
train masks: 2439
val images: 1041
val masks: 1041


In [1]:
import os
import cv2
import random

# === PATHS ===
base_dir = "/Users/ajayyy/Desktop/Deep_Learning/Smart-Quality-Inspection-System/data/processed"
images_dir = os.path.join(base_dir, "images2/val")
labels_dir = os.path.join(base_dir, "labels2/val")
output_dir = os.path.join(base_dir, "sanity_check_outputs2")

os.makedirs(output_dir, exist_ok=True)

num_samples = 10 

# === FUNCTION TO DRAW BOXES ===
def draw_yolo_boxes(img_path, label_path, save_path):
    img = cv2.imread(img_path)
    if img is None:
        return False

    h, w = img.shape[:2]
    if not os.path.exists(label_path):
        return False

    with open(label_path, "r") as f:
        lines = f.readlines()

    for line in lines:
        parts = line.strip().split()
        if len(parts) != 5:
            continue
        cls, x, y, bw, bh = map(float, parts)
        x1 = int((x - bw / 2) * w)
        y1 = int((y - bh / 2) * h)
        x2 = int((x + bw / 2) * w)
        y2 = int((y + bh / 2) * h)
        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 0, 255), 2)
        cv2.putText(img, f"defect", (x1, y1 - 5),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)

    cv2.imwrite(save_path, img)
    return True

# === RANDOMLY CHECK SOME LABELS ===
label_files = [f for f in os.listdir(labels_dir) if f.endswith(".txt")]
print(f"Found {len(label_files)} label files.")

sample_files = random.sample(label_files, min(num_samples, len(label_files)))

for lf in sample_files:
    label_path = os.path.join(labels_dir, lf)
    img_name = lf.replace(".txt", ".png")
    img_path = os.path.join(images_dir, img_name)
    if not os.path.exists(img_path):
        img_name = lf.replace(".txt", ".jpg")
        img_path = os.path.join(images_dir, img_name)
        if not os.path.exists(img_path):
            continue
    save_path = os.path.join(output_dir, img_name)
    success = draw_yolo_boxes(img_path, label_path, save_path)
    if success:
        print(f" Visualized: {img_name}")
    else:
        print(f"Skipped: {img_name}")

print(f"\nSanity check complete! Labeled samples saved to:\n{output_dir}")


Found 147 label files.
 Visualized: 20378_aug1769.png
 Visualized: 20821.png
 Visualized: 20669.png
 Visualized: 20172_aug8612.png
 Visualized: 20632_aug4385.png
 Visualized: 20682_aug6115.png
 Visualized: 20099.png
 Visualized: 20056.png
 Visualized: 20587.png
 Visualized: 20772.png

Sanity check complete! Labeled samples saved to:
/Users/ajayyy/Desktop/Deep_Learning/Smart-Quality-Inspection-System/data/processed/sanity_check_outputs2


## NEU-DET

In [5]:
import os

base_dir = "/Users/ajayyy/Desktop/Deep_Learning/Smart-Quality-Inspection-System/data/raw/NEU-DET"
splits = ["train", "validation"]

for split in splits:
    images_dir = os.path.join(base_dir, split, "images")
    labels_dir = os.path.join(base_dir, "labels", split)

    # Collect all image filenames (without extension), recursively through class subfolders
    image_files = []
    for root, _, files in os.walk(images_dir):
        for f in files:
            if f.lower().endswith(('.jpg', '.png')):
                image_files.append(os.path.splitext(f)[0])

    # Collect all label filenames (without extension)
    label_files = []
    if os.path.exists(labels_dir):
        for root, _, files in os.walk(labels_dir):
            for f in files:
                if f.endswith('.txt'):
                    label_files.append(os.path.splitext(f)[0])
    else:
        print(f" Labels folder does not exist for split: {split}")

    image_set = set(image_files)
    label_set = set(label_files)

    missing_labels = image_set - label_set
    extra_labels = label_set - image_set

    print(f"\n=== {split.upper()} SPLIT ===")
    print(f"Total images: {len(image_files)}")
    print(f"Total labels: {len(label_files)}")
    print(f"Images without labels: {len(missing_labels)}")
    print(f"Labels without images: {len(extra_labels)}")

    if missing_labels:
        print("Missing labels for images:", sorted(missing_labels))
    if extra_labels:
        print("Labels with no corresponding images:", sorted(extra_labels))



=== TRAIN SPLIT ===
Total images: 1440
Total labels: 1440
Images without labels: 0
Labels without images: 0

=== VALIDATION SPLIT ===
Total images: 360
Total labels: 360
Images without labels: 0
Labels without images: 0


In [8]:
import os
import cv2
import random

# === PATHS ===
base_dir = "/Users/ajayyy/Desktop/Deep_Learning/Smart-Quality-Inspection-System/data/raw/NEU-DET"
splits = ["train", "validation"]
output_dir = os.path.join(base_dir, "bbox_visuals")
os.makedirs(output_dir, exist_ok=True)

num_samples = 5  # images per split

# === FUNCTION TO DRAW YOLO BOXES ===
def draw_yolo_boxes(img_path, label_path, save_path):
    img = cv2.imread(img_path)
    if img is None:
        return False
    h, w = img.shape[:2]
    if not os.path.exists(label_path):
        return False
    with open(label_path, "r") as f:
        lines = f.readlines()
    for line in lines:
        parts = line.strip().split()
        if len(parts) != 5:
            continue
        cls_id, x, y, bw, bh = map(float, parts)
        x1 = int((x - bw / 2) * w)
        y1 = int((y - bh / 2) * h)
        x2 = int((x + bw / 2) * w)
        y2 = int((y + bh / 2) * h)
        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 0, 255), 2)
        cv2.putText(img, str(int(cls_id)), (x1, y1 - 5),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)
    cv2.imwrite(save_path, img)
    return True

# === HELPER FUNCTION TO FIND IMAGE RECURSIVELY ===
def find_image(image_name, search_dir):
    for root, _, files in os.walk(search_dir):
        for file in files:
            if file.startswith(image_name):
                return os.path.join(root, file)
    return None

# === VISUALIZE RANDOM SAMPLES ===
for split in splits:
    images_dir = os.path.join(base_dir, split)
    labels_dir = os.path.join(base_dir, "labels", split)
    split_output_dir = os.path.join(output_dir, split)
    os.makedirs(split_output_dir, exist_ok=True)

    label_files = [f for f in os.listdir(labels_dir) if f.endswith(".txt")]
    sample_files = random.sample(label_files, min(num_samples, len(label_files)))

    for lf in sample_files:
        label_path = os.path.join(labels_dir, lf)
        img_name = lf.replace(".txt", "")

        img_path = find_image(img_name, images_dir)
        if img_path is None:
            print(f"Image not found for label: {lf}")
            continue

        save_path = os.path.join(split_output_dir, os.path.basename(img_path))
        success = draw_yolo_boxes(img_path, label_path, save_path)
        if success:
            print(f" Visualized: {os.path.basename(img_path)}")
        else:
            print(f"Skipped: {os.path.basename(img_path)}")

print(f"\nBounding box visualization complete! Check the folder:\n{output_dir}")


 Visualized: pitted_surface_218.jpg
 Visualized: inclusion_146.jpg
 Visualized: pitted_surface_207.jpg
 Visualized: scratches_66.jpg
 Visualized: inclusion_107.jpg
 Visualized: pitted_surface_259.jpg
 Visualized: pitted_surface_272.jpg
 Visualized: scratches_255.jpg
 Visualized: pitted_surface_257.jpg
 Visualized: patches_281.jpg

Bounding box visualization complete! Check the folder:
/Users/ajayyy/Desktop/Deep_Learning/Smart-Quality-Inspection-System/data/raw/NEU-DET/bbox_visuals


In [6]:
%pip install opencv-python

  Using cached opencv_python-4.12.0.88-cp37-abi3-macosx_13_0_arm64.whl.metadata (19 kB)
  Using cached numpy-2.2.6-cp310-cp310-macosx_14_0_arm64.whl.metadata (62 kB)
Using cached opencv_python-4.12.0.88-cp37-abi3-macosx_13_0_arm64.whl (37.9 MB)
Using cached numpy-2.2.6-cp310-cp310-macosx_14_0_arm64.whl (5.3 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [opencv-python]0m [opencv-python]
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
from pathlib import Path
from collections import Counter, defaultdict
import cv2
import numpy as np

ROOT_IMG = Path("/Users/ajayyy/Desktop/Deep_Learning/Smart-Quality-Inspection/data/images")
ROOT_LAB = Path("/Users/ajayyy/Desktop/Deep_Learning/Smart-Quality-Inspection/data/labels")
EXTS = [".jpg", ".png", ".jpeg", ".bmp", ".tif"]

def find_images(root):
    imgs = []
    for p in root.rglob("*"):
        if p.suffix.lower() in EXTS:
            imgs.append(p)
    return imgs

def corresponding_label(img_path):
    rel = img_path.relative_to(ROOT_IMG)
    return ROOT_LAB / rel.with_suffix(".txt")

def read_label_file(label_path):
    if not label_path.exists():
        return None
    lines = [l.strip() for l in label_path.read_text().splitlines() if l.strip()]
    return lines

def check_yolo_line(line):
    parts = line.split()
    if len(parts) < 5:
        return None
    try:
        cls = int(parts[0])
        x, y, w, h = map(float, parts[1:5])
    except:
        return None
    if not (0 <= x <= 1 and 0 <= y <= 1 and 0 <= w <= 1 and 0 <= h <= 1):
        return None
    area = w * h
    return {"cls": cls, "area": area}

def analyze_dataset():
    splits = ["train", "val"]
    summary = {}

    for split in splits:
        print(f"\n=== Analyzing split: {split.upper()} ===")
        img_dir = ROOT_IMG / split
        lab_dir = ROOT_LAB / split

        imgs = find_images(img_dir)
        print(f"Found {len(imgs)} images under {img_dir}")

        empty_labels = 0
        missing_labels = 0
        bad_lines = 0
        total_boxes = 0
        class_counts = Counter()
        box_areas = []
        img_shapes = Counter()
        boxes_per_img = []

        for img_path in imgs:
            lab_path = corresponding_label(img_path)
            lines = read_label_file(lab_path)
            if lines is None:
                missing_labels += 1
                continue
            if len(lines) == 0:
                empty_labels += 1
                continue

            img = cv2.imread(str(img_path))
            if img is not None:
                h, w = img.shape[:2]
                img_shapes[(w, h)] += 1

            valid_boxes = 0
            for line in lines:
                res = check_yolo_line(line)
                if not res:
                    bad_lines += 1
                    continue
                class_counts[res["cls"]] += 1
                box_areas.append(res["area"])
                valid_boxes += 1
            total_boxes += valid_boxes
            boxes_per_img.append(valid_boxes)

        # Compute stats
        mean_boxes = np.mean(boxes_per_img) if boxes_per_img else 0
        mean_area = np.mean(box_areas) if box_areas else 0
        min_area = np.min(box_areas) if box_areas else 0
        max_area = np.max(box_areas) if box_areas else 0

        print(f"Missing label files: {missing_labels}")
        print(f"Empty label files: {empty_labels}")
        print(f"Bad label lines: {bad_lines}")
        print(f"Total boxes: {total_boxes}")
        print(f"Avg boxes per image: {mean_boxes:.2f}")
        print(f"Box area stats (normalized): min={min_area:.6f}, mean={mean_area:.6f}, max={max_area:.6f}")
        print("\nTop 10 most common image sizes (WxH):")
        for (w, h), c in img_shapes.most_common(10):
            print(f"  {w}x{h}: {c} images")

        print("\nClass distribution:")
        total_labels = sum(class_counts.values())
        for cls, cnt in class_counts.most_common():
            pct = (cnt / total_labels * 100) if total_labels else 0
            print(f"  class {cls}: {cnt} ({pct:.2f}%)")

        outliers_small = sum(a < 0.0001 for a in box_areas)
        outliers_large = sum(a > 0.5 for a in box_areas)
        print(f"\nOutlier boxes: too small={outliers_small}, too large={outliers_large}")
        print("=" * 40)

        summary[split] = {
            "images": len(imgs),
            "missing": missing_labels,
            "empty": empty_labels,
            "total_boxes": total_boxes,
            "mean_boxes": mean_boxes,
            "classes": dict(class_counts)
        }

    print("\n=== SUMMARY ===")
    for k, v in summary.items():
        print(f"{k.upper()}: {v['images']} imgs, {v['empty']} empty, {v['total_boxes']} boxes, mean boxes/img {v['mean_boxes']:.2f}")
    print("\nDone. Your dataset now has no excuses left.")

if __name__ == "__main__":
    analyze_dataset()



=== Analyzing split: TRAIN ===
Found 10907 images under /Users/ajayyy/Desktop/Deep_Learning/Smart-Quality-Inspection/data/images/train
Missing label files: 0
Empty label files: 5574
Bad label lines: 8832
Total boxes: 3336
Avg boxes per image: 0.63
Box area stats (normalized): min=0.000242, mean=0.155360, max=0.990025

Top 10 most common image sizes (WxH):
  200x200: 4980 images
  512x512: 353 images

Class distribution:
  class 0: 3336 (100.00%)

Outlier boxes: too small=0, too large=189

=== Analyzing split: VAL ===
Found 3126 images under /Users/ajayyy/Desktop/Deep_Learning/Smart-Quality-Inspection/data/images/val
Missing label files: 0
Empty label files: 1362
Bad label lines: 0
Total boxes: 2886
Avg boxes per image: 1.64
Box area stats (normalized): min=0.000076, mean=0.097044, max=0.974609

Top 10 most common image sizes (WxH):
  512x512: 1404 images
  200x200: 360 images

Class distribution:
  class 0: 2886 (100.00%)

Outlier boxes: too small=8, too large=110

=== SUMMARY ===
TRA

In [ ]:
%pip install -r /Users/ajayyy/Desktop/Deep_Learning/Smart-Quality-Inspection/requirements.txt

  Cloning https://github.com/facebookresearch/segment-anything.git to /private/var/folders/sg/mjc3zgxd0xggmd4mpy2fkrq80000gn/T/pip-req-build-aevqx5d1
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /private/var/folders/sg/mjc3zgxd0xggmd4mpy2fkrq80000gn/T/pip-req-build-aevqx5d1
  Resolved https://github.com/facebookresearch/segment-anything.git to commit dca509fe793f601edb92606367a655c15ac00fdf
  Preparing metadata (setup.py) ... done
  Using cached ultralytics-8.3.226-py3-none-any.whl.metadata (37 kB)
  Using cached segment_anything-1.0-py3-none-any.whl.metadata (487 bytes)
  Using cached uvicorn-0.38.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached streamlit-1.51.0-py3-none-any.whl.metadata (9.5 kB)
  Using cached docker-7.1.0-py3-none-any.whl.metadata (3.8 kB)
  Using cached psycopg2-2.9.11.tar.gz (379 kB)
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info d

In [6]:
%pip install albumentations

  Using cached albumentations-2.0.8-py3-none-any.whl.metadata (43 kB)
  Using cached scipy-1.15.3-cp310-cp310-macosx_14_0_arm64.whl.metadata (61 kB)
  Using cached albucore-0.0.24-py3-none-any.whl.metadata (5.3 kB)
  Using cached opencv_python_headless-4.12.0.88-cp37-abi3-macosx_13_0_arm64.whl.metadata (19 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached typing_inspection-0.4.2-py3-none-any.whl.metadata (2.6 kB)
Using cached albumentations-2.0.8-py3-none-any.whl (369 kB)
Using cached albucore-0.0.24-py3-none-any.whl (15 kB)
Using cached opencv_python_headless-4.12.0.88-cp37-abi3-macosx_13_0_arm64.whl (37.9 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 1.8 MB/s  0:00:00 eta 0:00:01
Using cached annotated_types-0.7.0-py3-none-any.whl (13 kB)
Using cached scipy-1.15.3-cp310-cp310-macosx_14_0_arm64.whl (22.4 MB)
Using cached typing_inspection-0.4.2-py3-none-any.whl (14 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11/11 [albumentation

### Balancing the training dataset to 60:40 non-defect to defect ratio

In [10]:
import os
import cv2
import math
import random
from glob import glob
import albumentations as A
from tqdm import tqdm

# ==== CONFIG ====
IMG_DIR = "/Users/ajayyy/Desktop/Deep_Learning/Smart-Quality-Inspection/data/images/train/"
LBL_DIR = "/Users/ajayyy/Desktop/Deep_Learning/Smart-Quality-Inspection/data/labels/train/"
OUT_IMG = "/Users/ajayyy/Desktop/Deep_Learning/Smart-Quality-Inspection/data/images/train_aug/"
OUT_LBL = "/Users/ajayyy/Desktop/Deep_Learning/Smart-Quality-Inspection/data/labels/train_aug/"
TARGET_RATIO = 0.6  # good:defect ratio

os.makedirs(OUT_IMG, exist_ok=True)
os.makedirs(OUT_LBL, exist_ok=True)

# ==== AUGMENTATION PIPELINE ====
transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.3),
    A.Rotate(limit=15, p=0.4),
    A.MotionBlur(p=0.2),
    A.GaussNoise(p=0.2),
], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels'], min_visibility=0.2))

# ==== STEP 1: Count good vs defect images ====
label_files = glob(os.path.join(LBL_DIR, "*.txt"))
good_imgs, defect_imgs = [], []

for lbl_path in label_files:
    with open(lbl_path) as f:
        lines = [ln.strip() for ln in f.readlines() if ln.strip()]
    if len(lines) == 0:
        good_imgs.append(lbl_path)
    else:
        defect_imgs.append(lbl_path)

good_count = len(good_imgs)
defect_count = len(defect_imgs)

print(f"\n=== BEFORE AUGMENTATION ===")
print(f"Good images:   {good_count}")
print(f"Defect images: {defect_count}")

# ==== STEP 2: Calculate how many augmentations needed ====
target_defect_count = int(good_count * (1 - TARGET_RATIO) / TARGET_RATIO)
augmentations_per_image = max(1, math.ceil(target_defect_count / defect_count))

print(f"\nTarget defect count for {TARGET_RATIO:.0%} ratio: {target_defect_count}")
print(f"Each defect image will be augmented {augmentations_per_image} times.")

# ==== STEP 3: Augment defect images ====
for lbl_path in tqdm(defect_imgs, desc="Augmenting defect images"):
    img_path = os.path.join(IMG_DIR, os.path.basename(lbl_path).replace(".txt", ".jpg"))
    if not os.path.exists(img_path):
        continue

    with open(lbl_path) as f:
        lines = [ln.strip() for ln in f.readlines() if ln.strip()]

    # Parse YOLO-format boxes
    boxes, classes = [], []
    for line in lines:
        cls, x, y, w, h = map(float, line.split())
        boxes.append([x, y, w, h])
        classes.append(int(cls))

    image = cv2.imread(img_path)
    if image is None:
        continue

    for i in range(augmentations_per_image):
        augmented = transform(image=image, bboxes=boxes, class_labels=classes)
        aug_img = augmented['image']
        aug_bboxes = augmented['bboxes']
        aug_classes = augmented['class_labels']

        # Save augmented image and label
        base = os.path.basename(img_path).replace(".jpg", f"_aug{i}.jpg")
        cv2.imwrite(os.path.join(OUT_IMG, base), aug_img)

        with open(os.path.join(OUT_LBL, base.replace(".jpg", ".txt")), "w") as f:
            for cls, (x, y, w, h) in zip(aug_classes, aug_bboxes):
                f.write(f"{cls} {x:.6f} {y:.6f} {w:.6f} {h:.6f}\n")

# ==== STEP 4: Final summary ====
augmented_count = len(glob(os.path.join(OUT_IMG, "*.jpg")))
final_defect_count = defect_count + augmented_count
total_images = good_count + final_defect_count
final_ratio = good_count / total_images

print(f"\n=== AFTER AUGMENTATION ===")
print(f"Good images:   {good_count}")
print(f"Defect images: {final_defect_count}")
print(f"Total images:  {total_images}")
print(f"Final good:defect ratio = {final_ratio:.2%}:{1 - final_ratio:.2%}")

print("\nBalanced dataset created successfully!")
print(f"Augmented images saved under: {OUT_IMG}")



=== BEFORE AUGMENTATION ===
Good images:   5574
Defect images: 1598

Target defect count for 60% ratio: 3716
Each defect image will be augmented 3 times.


Augmenting defect images: 100%|██████████| 1598/1598 [00:05<00:00, 282.34it/s]


=== AFTER AUGMENTATION ===
Good images:   5574
Defect images: 5333
Total images:  10907
Final good:defect ratio = 51.10%:48.90%

Balanced dataset created successfully!
Augmented images saved under: /Users/ajayyy/Desktop/Deep_Learning/Smart-Quality-Inspection/data/images/train_aug/


In [8]:
%pip install tdqm

  Using cached tdqm-0.0.1.tar.gz (1.4 kB)
  Preparing metadata (setup.py) ... done
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
  DEPRECATION: Building 'tdqm' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'tdqm'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  Created wheel for tdqm: filename=tdqm-0.0.1-py3-none-any.whl size=1384 sha256=d515acedc253106246f3883eaac0a012caf6550f3ad3a1bf07ae294caa86e1e7
  Stored in directory: /Users/ajayyy/Library/Caches/pip/wheels/37/31/b8/7b711038035720ba0df14376af06e5e76b9bd61759c861ad92
Successfully built tdqm
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [tdqm]
Note: y

In [12]:
import os
from glob import glob

label_dir = "/Users/ajayyy/Desktop/Deep_Learning/Smart-Quality-Inspection/data/labels/train/"
bad_files = []

for lbl_path in glob(os.path.join(label_dir, "*.txt")):
    with open(lbl_path) as f:
        for i, line in enumerate(f, 1):
            vals = line.strip().split()
            # catch empty or malformed lines
            if len(vals) != 5:
                bad_files.append((lbl_path, i, line.strip(), "wrong number of values"))
                continue
            try:
                cls, x, y, w, h = map(float, vals)
            except ValueError:
                bad_files.append((lbl_path, i, line.strip(), "non-numeric value"))
                continue
            # catch coordinates outside normalized range
            if not (0 <= x <= 1 and 0 <= y <= 1 and 0 < w <= 1 and 0 < h <= 1):
                bad_files.append((lbl_path, i, line.strip(), "out-of-range coordinates"))

# print summary
print(f"Total suspicious lines: {len(bad_files)}\n")
for path, line_no, content, reason in bad_files[:20]:  # show first 20 only
    print(f"{os.path.basename(path)} (line {line_no}): {reason} → {content}")

if len(bad_files) > 20:
    print(f"...and {len(bad_files) - 20} more.")


Total suspicious lines: 0



In [ ]:
import os
import cv2
from pathlib import Path

# === CONFIG ===
IMG_DIR = Path("/Users/ajayyy/Desktop/Deep_Learning/Smart-Quality-Inspection/data/images/train")
LBL_DIR = Path("/Users/ajayyy/Desktop/Deep_Learning/Smart-Quality-Inspection/data/labels/train")
IMG_EXTS = [".jpg", ".png", ".jpeg"]

# === FUNCTION TO DRAW BOXES ===
def draw_boxes(image_path, label_path):
    img = cv2.imread(str(image_path))
    if img is None:
        print(f"[SKIP] Could not read {image_path.name}")
        return None

    h, w, _ = img.shape
    with open(label_path, "r") as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) != 5:
                print(f"[BAD LINE] {label_path.name}: {line.strip()}")
                continue

            cls, x, y, bw, bh = map(float, parts)
            # YOLO format -> convert to pixel coordinates
            x1 = int((x - bw / 2) * w)
            y1 = int((y - bh / 2) * h)
            x2 = int((x + bw / 2) * w)
            y2 = int((y + bh / 2) * h)

            # Draw rectangle
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 0, 255), 2)
            cv2.putText(img, str(int(cls)), (x1, y1 - 5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

    return img

# === MAIN LOOP ===
for img_file in IMG_DIR.glob("*"):
    if img_file.suffix.lower() not in IMG_EXTS:
        continue

    label_file = LBL_DIR / (img_file.stem + ".txt")
    if not label_file.exists():
        print(f"[MISSING LABEL] {img_file.name}")
        continue

    img_with_boxes = draw_boxes(img_file, label_file)
    if img_with_boxes is not None:
        cv2.imshow("Bounding Boxes", img_with_boxes)
        key = cv2.waitKey(0)
        if key == 27:  # ESC to quit early
            break

cv2.destroyAllWindows()


[SKIP] Could not read 10251_aug1.png
[SKIP] Could not read 11259_aug3.png
[SKIP] Could not read 10778_aug3.png
[SKIP] Could not read 11267_aug2.png
[SKIP] Could not read 12071_aug1.png
[SKIP] Could not read 10416_aug3.png
[SKIP] Could not read 10193_aug1.png
[SKIP] Could not read 10192_aug1.png
[SKIP] Could not read 11210_aug2.png
[SKIP] Could not read 12006_aug1.png
[SKIP] Could not read 10485_aug2.png
[SKIP] Could not read 10772_aug1.png
[SKIP] Could not read 10773_aug1.png
[SKIP] Could not read 10074_aug3.png
[SKIP] Could not read 10922_aug3.png
[SKIP] Could not read 11486_aug3.png
[SKIP] Could not read 11487_aug3.png
[SKIP] Could not read 12229_aug3.png
[SKIP] Could not read 10586_aug3.png


In [2]:
import os
from glob import glob

label_dir = "/Users/ajayyy/Desktop/Deep_Learning/Smart-Quality-Inspection/data/labels/train/"
bad_files = []

for lbl_path in glob(os.path.join(label_dir, "*.txt")):
    with open(lbl_path, "r", encoding="utf-8", errors="ignore") as f:
        lines = f.readlines()

    if not lines:
        bad_files.append((lbl_path, "empty file"))
        continue

    for i, line in enumerate(lines, 1):
        line_clean = line.strip()
        if not line_clean:
            bad_files.append((lbl_path, f"blank line at {i}"))
            continue

        parts = line_clean.split()
        if len(parts) != 5:
            bad_files.append((lbl_path, f"line {i} → wrong number of values ({len(parts)})"))
            continue

        try:
            cls, x, y, w, h = map(float, parts)
        except ValueError:
            bad_files.append((lbl_path, f"line {i} → non-numeric value"))
            continue

        if not (0 <= x <= 1 and 0 <= y <= 1 and 0 < w <= 1 and 0 < h <= 1):
            bad_files.append((lbl_path, f"line {i} → out-of-range coordinates"))

# Print summary
if bad_files:
    print(f"Total bad label files: {len(set(f[0] for f in bad_files))}\n")
    for file, issue in bad_files[:30]:
        print(f"{os.path.basename(file)} → {issue}")
    if len(bad_files) > 30:
        print(f"...and {len(bad_files) - 30} more issues.")
else:
    print("✅ All label files look fine!")


Total bad label files: 5574

pill_good_train_008_aug1.txt → empty file
10203_aug3.txt → empty file
11871_aug2.txt → empty file
metal_nut_good_train_145_aug3.txt → empty file
capsule_good_train_083_aug3.txt → empty file
screw_good_train_017_aug3.txt → empty file
11870_aug2.txt → empty file
10064_aug3.txt → empty file
11471.txt → empty file
screw_good_train_016_aug3.txt → empty file
10009.txt → empty file
10747.txt → empty file
11405_aug3.txt → empty file
12186_aug2.txt → empty file
12144.txt → empty file
screw_good_train_294_aug2.txt → empty file
capsule_good_train_066_aug2.txt → empty file
bottle_good_train_069_aug1.txt → empty file
pill_good_train_192_aug1.txt → empty file
transistor_good_train_211_aug2.txt → empty file
screw_good_train_245_aug1.txt → empty file
tile_good_train_215_aug1.txt → empty file
wood_good_train_193_aug3.txt → empty file
leather_good_train_222_aug2.txt → empty file
pill_good_train_193_aug1.txt → empty file
leather_good_train_044_aug2.txt → empty file
transistor